In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [3]:
sc.stop()

a) Create a new Spark Session with new SparkConfig

In [4]:
from pyspark import SparkConf, SparkContext
# setMaster() - set spark context manager which is local[cpu_cores]
config = SparkConf().setMaster("local[4]").setAppName("PySparkSession")
sc = SparkContext(conf=config)

In [5]:
sc

<SparkContext master=local[4] appName=PySparkSession>

b) Create new instance of Spark SQL session and define new DataFrame using
sales_data_sample.csv dataset.

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQLSession").getOrCreate()

In [7]:
spark

In [3]:
sales_df = spark.read.csv("file:///home/hadoop/Downloads/sales_data_sample.csv",header=True,inferSchema=True)

In [4]:
sales_df.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

c) Find the shape of DataFrame.

In [5]:
# Number of rows
sales_df.count()

2823

In [6]:
# Number of columns
len(sales_df.columns)

25

d) Find the Summary of DataFrame for all numerical data columns.

In [7]:
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import *

In [8]:
numerical_cols = [field.name for field in sales_df.schema.fields if not isinstance(field.dataType, StringType)]

In [9]:
sales_df[numerical_cols].summary().show()

+-------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|       ORDERNUMBER|  QUANTITYORDERED|         PRICEEACH|  ORDERLINENUMBER|             SALES|            QTR_ID|          MONTH_ID|           YEAR_ID|              MSRP|
+-------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|              2823|             2823|              2823|             2823|              2823|              2823|              2823|              2823|              2823|
|   mean|10258.725115125753|35.09280906836698| 83.65854410201929|6.466170740347148|  3553.88907190932|2.7176762309599716|7.0924548352816155|2003.8150903294368|100.71555083244775|
| stddev|  92.0854775957196| 9.74144273706958|20.174276527840536| 4.22584096469094|1841.8651057401842| 1.

e) Identify and handle missing or null values in the columns.

In [10]:
from pyspark.sql.functions import *

sales_df.select([sum(col(column).isNull().cast('int')).alias(column) for column in sales_df.columns]).show()

+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|ADDRESSLINE2|CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|          0|              0|        0|              0|    0|        0|     0|     0|       0|      0|          0|   0|          0|           0|    0|           0|        2521|   0| 1486|        76|      0|        0|       

In [11]:
sales_df1 = sales_df.fillna("Nill")

In [12]:
from pyspark.sql.functions import *

sales_df1.select([sum(col(column).isNull().cast('int')).alias(column) for column in sales_df1.columns]).show()

+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|ADDRESSLINE2|CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+------+------+--------+-------+-----------+----+-----------+------------+-----+------------+------------+----+-----+----------+-------+---------+---------------+----------------+--------+
|          0|              0|        0|              0|    0|        0|     0|     0|       0|      0|          0|   0|          0|           0|    0|           0|           0|   0|    0|         0|      0|        0|       

f) Calculate the total revenue generated per country by combining the columns
QUANTITYORDERED and PRICEEACH using Spark DataFrame operations?

In [13]:
sales_df2 = sales_df1.withColumn('Total Revenue',col('QUANTITYORDERED')*col('PRICEEACH'))
sales_df2.groupBy('COUNTRY').agg(sum('Total Revenue').alias('Total Revenue')).show()

+-----------+------------------+
|    COUNTRY|     Total Revenue|
+-----------+------------------+
|     Sweden|174264.10000000006|
|Philippines| 80291.16999999998|
|  Singapore| 227985.5000000001|
|    Germany|         178689.08|
|     France| 919257.8499999997|
|    Belgium|          94528.88|
|    Finland|268714.70000000007|
|      Italy| 309402.8699999999|
|     Norway| 246115.8000000001|
|      Spain|1021705.9700000002|
|    Denmark|         192747.63|
|    Ireland|          43237.24|
|        USA|2986425.2099999995|
|         UK|413203.33999999997|
|Switzerland| 93344.90999999999|
|     Canada|193504.34000000003|
|      Japan|153076.68999999994|
|  Australia|521598.45999999985|
|    Austria|172793.05000000002|
+-----------+------------------+



g) Determine the top 5 products with the highest total sales revenue using Spark DataFrame?

In [14]:
product_sales = sales_df1.groupBy("PRODUCTLINE").agg(sum("SALES").alias("total_sales"))


sorted_sales = product_sales.orderBy(col("total_sales").desc())
top_5_products = sorted_sales.limit(5)
top_5_products.show()

+----------------+------------------+
|     PRODUCTLINE|       total_sales|
+----------------+------------------+
|    Classic Cars| 3919615.659999997|
|    Vintage Cars|1903150.8399999992|
|     Motorcycles|1166388.3400000003|
|Trucks and Buses|1127789.8399999996|
|          Planes| 975003.5700000001|
+----------------+------------------+



h) Find the average order quantity for each product using groupBy and agg operations?

In [15]:
sales_df1.groupBy('PRODUCTLINE').agg(mean('QUANTITYORDERED').alias('Avg order quantity')).show()

+----------------+------------------+
|     PRODUCTLINE|Avg order quantity|
+----------------+------------------+
|     Motorcycles|35.235649546827794|
|    Vintage Cars| 34.71004942339374|
|           Ships| 34.73076923076923|
|Trucks and Buses| 35.80398671096346|
|    Classic Cars|35.152016546018615|
|          Trains| 35.22077922077922|
|          Planes| 35.05555555555556|
+----------------+------------------+



i) Using Spark DataFrame, filter orders where the SALES value exceeds $10,000 and sort the
results by the ORDERDATE column?

In [16]:
sales_df1 = sales_df1.withColumn("ORDERDATE", to_timestamp(col("ORDERDATE"), "M/d/yyyy H:mm"))
sales_df1.filter(sales_df1.SALES > 10000).orderBy(sales_df1.ORDERDATE).show()

+-----------+---------------+---------+---------------+-------+-------------------+----------+------+--------+-------+------------+----+-----------+--------------------+---------------+--------------------+------------+-------------+-----+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|          ORDERDATE|    STATUS|QTR_ID|MONTH_ID|YEAR_ID| PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|          PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+-------------------+----------+------+--------+-------+------------+----+-----------+--------------------+---------------+--------------------+------------+-------------+-----+----------+---------+---------+---------------+----------------+--------+
|      10127|             46|    100.0|              2|11279.

j) Filter out rows where the STATUS is &#39;Cancelled&#39; and calculate the total sales from the
remaining orders?

In [17]:
sales_df1.filter(sales_df1.STATUS != 'Cancelled').agg(sum('SALES').alias('Total Sales')).show()

+-----------------+
|      Total Sales|
+-----------------+
|9838141.370000018|
+-----------------+



k) Use Spark Data Frame transformations to derive the yearly sales for each customer
(CUSTOMERNAME) based on the ORDERDATE column?

In [18]:
from pyspark.sql.functions import *
sales_df1 = sales_df1.withColumn("ORDERDATE", to_timestamp(col("ORDERDATE"), "M/d/yyyy H:mm"))

sales_df1 = sales_df1.withColumn("YEAR", year(col("ORDERDATE")))


yearly_sales_df = sales_df1.groupBy("CUSTOMERNAME", "YEAR").agg(sum("SALES").alias("TOTAL_SALES"))

yearly_sales_df.show()


+--------------------+----+------------------+
|        CUSTOMERNAME|YEAR|       TOTAL_SALES|
+--------------------+----+------------------+
|  Baane Mini Imports|2003|56176.659999999996|
|Stylish Desk Deco...|2004|13739.900000000001|
|Marseille Mini Autos|2003|52481.840000000004|
|Danish Wholesale ...|2004|          60157.62|
|Toms Spezialitten...|2003|          31363.18|
|Australian Collec...|2004|140859.56999999998|
|Dragon Souveniers...|2004|           3127.88|
|    Super Scale Inc.|2003|          42498.76|
|Collectables For ...|2004|15110.800000000001|
|Royal Canadian Co...|2004| 74634.84999999999|
|Online Diecast Cr...|2003|           76114.7|
|   Gifts4AllAges.com|2005|          48316.89|
|        Herkku Gifts|2004|           16363.1|
|Diecast Classics ...|2004|115971.34000000001|
|Motor Mint Distri...|2003|          27398.82|
|Daedalus Designs ...|2003|48874.280000000006|
|Stylish Desk Deco...|2003|           75064.6|
|         Mini Caravy|2005|          35680.35|
|     Mini Wh

l) Add a new column to the DataFrame that categorizes orders as &quot;High&quot;, &quot;Medium&quot;, or &quot;Low&quot;
sales based on the SALES value?

In [71]:
quantiles = sales_df1.approxQuantile('SALES',[0.33,0.67],0.0001)
val1 = quantiles[0]
val2 = quantiles[1]
                                                       
sales_df1 = sales_df1.withColumn('CATEGORY',when(col('SALES') <= val1,'Low').when(col('SALES') <= val2,'Medium')
                                 .otherwise('High'))  
sales_df1.show()

+-----------+---------------+---------+---------------+-------+-------------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+----+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|          ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|YEAR|CATEGORY|
+-----------+---------------+---------+---------------+-------+-------------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+----+--------+
|      10107|      

m) Assume , If you have another DataFrame with customer demographic data, how would you
perform a join to compute the total sales per demographic group?

In [101]:
customer_demographics_data = [
    ("Land of Toys Inc.", "25-34", "High", "VIP"),
    ("Reims Collectables", "35-44", "Medium", "Regular"),
    ("Lyon Souveniers", "45-54", "Medium", "Regular"),
    ("Toys4GrownUps.com", "25-34", "High", "VIP"),
    ("Corporate Gift Ideas", "35-44", "High", "VIP"),
    ("Technics Stores Inc.", "45-54", "Medium", "Regular"),
    ("Daedalus Designs Inc.", "55-64", "Low", "New"),
    ("Herkku Gifts", "65+", "Norway", "New"),
    ("Mini Wheels Co.", "25-34", "Medium", "Regular"),
    ("Auto Canal Petit", "35-44", "Medium", "Regular"),
]
customer_demographics_columns = ["CUSTOMERNAME", "AGE_GROUP", "INCOME_LEVEL", "LOYALTY_STATUS"]


customer_demographics_df = spark.createDataFrame(customer_demographics_data, schema=customer_demographics_columns)

customer_demographics_df.show()


+--------------------+---------+------------+--------------+
|        CUSTOMERNAME|AGE_GROUP|INCOME_LEVEL|LOYALTY_STATUS|
+--------------------+---------+------------+--------------+
|   Land of Toys Inc.|    25-34|        High|           VIP|
|  Reims Collectables|    35-44|      Medium|       Regular|
|     Lyon Souveniers|    45-54|      Medium|       Regular|
|   Toys4GrownUps.com|    25-34|        High|           VIP|
|Corporate Gift Ideas|    35-44|        High|           VIP|
|Technics Stores Inc.|    45-54|      Medium|       Regular|
|Daedalus Designs ...|    55-64|         Low|           New|
|        Herkku Gifts|      65+|      Norway|           New|
|     Mini Wheels Co.|    25-34|      Medium|       Regular|
|    Auto Canal Petit|    35-44|      Medium|       Regular|
+--------------------+---------+------------+--------------+



In [102]:
from pyspark.sql.functions import sum


joined_df = sales_df.join(customer_demographics_df, on="CUSTOMERNAME")


total_sales_per_demographic_df = joined_df.groupBy("AGE_GROUP","CUSTOMERNAME", "INCOME_LEVEL", "LOYALTY_STATUS").agg(
    sum("SALES").alias("TOTAL_SALES")
)


total_sales_per_demographic_df.show()

+---------+--------------------+------------+--------------+------------------+
|AGE_GROUP|        CUSTOMERNAME|INCOME_LEVEL|LOYALTY_STATUS|       TOTAL_SALES|
+---------+--------------------+------------+--------------+------------------+
|    35-44|    Auto Canal Petit|      Medium|       Regular| 93170.66000000002|
|    45-54|     Lyon Souveniers|      Medium|       Regular|          78570.34|
|    35-44|  Reims Collectables|      Medium|       Regular|         135042.94|
|    25-34|   Toys4GrownUps.com|        High|           VIP|104561.95999999998|
|    25-34|     Mini Wheels Co.|      Medium|       Regular|          74476.18|
|      65+|        Herkku Gifts|      Norway|           New|         111640.28|
|    45-54|Technics Stores Inc.|      Medium|       Regular|120783.06999999999|
|    25-34|   Land of Toys Inc.|        High|           VIP|164069.43999999994|
+---------+--------------------+------------+--------------+------------------+



n) Can you implement a cumulative distribution function (CDF) over the SALES value for each
CUSTOMERNAME? What insights can you gather from analyzing the CDF distribution for each
customer?

In [138]:
window_spec = Window.partitionBy('CUSTOMERNAME').orderBy('SALES')

rank_df = sales_df1.withColumn('rank', row_number().over(window_spec))
count_df = rank_df.withColumn('total_count', count('SALES').over(Window.partitionBy('CUSTOMERNAME')))
cdf_df = count_df.withColumn('CDF', col('rank') / col('total_count'))
cdf_df1 = cdf_df.select('CUSTOMERNAME', 'SALES', 'CDF').show(50)

+-------------------+-------+--------------------+
|       CUSTOMERNAME|  SALES|                 CDF|
+-------------------+-------+--------------------+
|Suominen Souveniers| 891.03| 0.03333333333333333|
|Suominen Souveniers| 1086.6| 0.06666666666666667|
|Suominen Souveniers|1103.76|                 0.1|
|Suominen Souveniers|1629.04| 0.13333333333333333|
|Suominen Souveniers| 1988.4| 0.16666666666666666|
|Suominen Souveniers|2140.11|                 0.2|
|Suominen Souveniers|2447.76| 0.23333333333333334|
|Suominen Souveniers|2632.89| 0.26666666666666666|
|Suominen Souveniers| 2773.8|                 0.3|
|Suominen Souveniers|2775.08|  0.3333333333333333|
|Suominen Souveniers|2817.87| 0.36666666666666664|
|Suominen Souveniers|2851.84|                 0.4|
|Suominen Souveniers|2931.98| 0.43333333333333335|
|Suominen Souveniers|3128.65|  0.4666666666666667|
|Suominen Souveniers|3288.82|                 0.5|
|Suominen Souveniers|3595.62|  0.5333333333333333|
|Suominen Souveniers|3686.54|  

#### Insights
* CDF values show how sales are distributed within each customer group.
* Sales Concentration for a customer like "Suominen Souveniers," we can see that a significant portion of the total sales is concentrated at higher values. 
* For example, 0.7 CDF corresponds to a sales value of 4381.25, meaning that 70% of the sales data falls below this amount.

o) Write spark dataframe code to rank products by total revenue within each country
(COUNTRY)?

In [20]:
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

window_spec = Window.partitionBy("COUNTRY").orderBy(col("Total Revenue").desc())


ranked_df = sales_df2.withColumn("RANK", rank().over(window_spec))


ranked_df.select(col('PRODUCTCODE'),col('COUNTRY'),col('Total revenue'),col('RANK')).show()

+-----------+-------+------------------+----+
|PRODUCTCODE|COUNTRY|     Total revenue|RANK|
+-----------+-------+------------------+----+
|   S18_4600| Sweden|            4900.0|   1|
|   S18_4600| Sweden|            4800.0|   2|
|   S24_2300| Sweden|            4800.0|   2|
|   S12_4675| Sweden|            4700.0|   4|
|   S18_2949| Sweden|            4700.0|   4|
|   S18_2319| Sweden|            4600.0|   6|
|   S24_1578| Sweden|            4500.0|   7|
|   S10_4757| Sweden|            4400.0|   8|
|   S18_4522| Sweden|            4300.5|   9|
|   S18_1662| Sweden|            4300.0|  10|
|   S24_2300| Sweden|            4200.0|  11|
|   S12_1666| Sweden|            4100.0|  12|
|   S18_1097| Sweden|            4100.0|  12|
|   S24_2011| Sweden|            4100.0|  12|
|   S24_2000| Sweden|3988.6000000000004|  15|
|   S18_2625| Sweden|            3900.0|  16|
|   S18_1889| Sweden|3881.7799999999997|  17|
|   S10_1949| Sweden|            3700.0|  18|
|   S24_3151| Sweden|           35

p) Calculate a running total of SALES for each customer and show the top 5 customers by this
cumulative total?


In [93]:
window_spec = Window.partitionBy("CUSTOMERNAME").orderBy("ORDERDATE").rowsBetween(Window.unboundedPreceding, Window.currentRow)

running_total_df = sales_df1.withColumn("RUNNING_TOTAL", sum("SALES").over(window_spec))
per_customer_df = running_total_df.groupBy("CUSTOMERNAME").agg(
    sum("RUNNING_TOTAL").alias("TOTAL_CUMULATIVE_SALES")

top_customers_df = per_customer_df.orderBy(col("TOTAL_CUMULATIVE_SALES").desc()).limit(5)
top_customers_df.show()

+--------------------+----------------------+
|        CUSTOMERNAME|TOTAL_CUMULATIVE_SALES|
+--------------------+----------------------+
|Euro Shopping Cha...|  1.1636376556000003E8|
|Mini Gifts Distri...|   5.858921097000004E7|
|Australian Collec...|     5550391.160000001|
|  Muscle Machine Inc|     5175832.140000001|
|   La Rochelle Gifts|     4766253.330000002|
+--------------------+----------------------+



q) Identify and handle Outliers in DataFrame.

In [21]:
from pyspark.sql.functions import col

quantiles = sales_df.approxQuantile("SALES", [0.25, 0.75], 0.001)

Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1



lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


outliers_df = sales_df1.filter((col("SALES") < lower_bound) | (col("SALES") > upper_bound))
outliers_df.show()


handled_df = sales_df1.filter((col("SALES") >= lower_bound) & (col("SALES") <= upper_bound))


handled_df.show()

+-----------+---------------+---------+---------------+-------+-------------------+----------+------+--------+-------+------------+----+-----------+--------------------+--------------+--------------------+------------+--------------+----------+----------+---------+---------+---------------+----------------+--------+----+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|          ORDERDATE|    STATUS|QTR_ID|MONTH_ID|YEAR_ID| PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|         PHONE|        ADDRESSLINE1|ADDRESSLINE2|          CITY|     STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|YEAR|
+-----------+---------------+---------+---------------+-------+-------------------+----------+------+--------+-------+------------+----+-----------+--------------------+--------------+--------------------+------------+--------------+----------+----------+---------+---------+---------------+----------------+--------+----+
|      10150|             45|  

r) How would you cache a DataFrame containing sales data from the top 10 countries by sales
to avoid recomputation in subsequent transformations? What persistence level (e.g.
MEMORY_ONLY, MEMORY_AND_DISK) would you choose and why?

In [106]:
from pyspark.storagelevel import StorageLevel
sales_country = sales_df1.groupBy("COUNTRY").agg(sum("SALES")
                                        .alias("TOTAL_SALES")).orderBy("TOTAL_SALES", ascending=False).limit(10)
sales_country.show(5)
sales_country.cache()
sales_country.persist(StorageLevel.MEMORY_AND_DISK)

+---------+------------------+
|  COUNTRY|       TOTAL_SALES|
+---------+------------------+
|      USA|        3627982.83|
|    Spain|1215686.9200000009|
|   France|1110916.5199999993|
|Australia| 630623.1000000001|
|       UK| 478880.4600000001|
+---------+------------------+
only showing top 5 rows



DataFrame[COUNTRY: string, TOTAL_SALES: double]

#### Recommended Persistence Level

For caching the DataFrame of sales data from the top 10 countries, I would recommend using MEMORY_AND_DISK. This choice is suitable because:

    Data Size: The DataFrame might be relatively small after filtering to the top 10 countries, so it can mostly fit in memory.
    Resilience: If the DataFrame size grows or if you face memory constraints, it will spill to disk without recomputation.
    Performance Balance: Provides a good balance between performance (memory access) and reliability (disk spill).

s) How would you pivot the data to show PRODUCTLINE as columns and the total SALES for
each ORDERDATE as the values? What are the implications of pivoting large datasets in Spark?

In [141]:
pivot_df = sales_df1.groupBy("ORDERDATE", "PRODUCTLINE").agg(sum("SALES").alias("TOTAL_SALES"))
pivot_df.groupBy("ORDERDATE").pivot("PRODUCTLINE").sum("TOTAL_SALES").show()



+-------------------+------------------+------------------+------------------+------------------+-------+----------------+------------+
|          ORDERDATE|      Classic Cars|       Motorcycles|            Planes|             Ships| Trains|Trucks and Buses|Vintage Cars|
+-------------------+------------------+------------------+------------------+------------------+-------+----------------+------------+
|2005-03-02 00:00:00|              null|            4175.6|              null|              null|   null|            null|        null|
|2004-11-09 00:00:00|              null|              null|              null|           6673.29|3807.68|            null|     9665.35|
|2005-05-03 00:00:00|25040.629999999997|              null|              null|              null|   null|        27247.11|        null|
|2003-09-11 00:00:00| 43593.54000000001|              null|              null|              null|   null|            null|     3598.22|
|2005-04-01 00:00:00|           9661.44|        

#### Implications of Pivoting Large Datasets

    * Memory Usage: Pivoting reshapes data, creating many new columns for each unique pivot value, 
        which can lead to high memory consumption.

    * Performance: The operation can be slow with large datasets or many unique pivot values due to 
        increased computational complexity.

    * Column Explosion: A large number of unique pivot values can result in a DataFrame with many columns, 
        making it unwieldy and hard to manage.

    * Data Skew: Uneven distribution of data (e.g., some dates having much more sales) can cause 
        performance issues due to uneven partitioning.

    * Shuffling: Pivoting involves shuffling data across nodes, which can be expensive in 
        terms of time and resources.

    * Disk I/O: If the data exceeds memory capacity, Spark will spill data to disk, 
        potentially slowing down the process.

t) How would you calculate the percentage growth of total sales month over month for each
PRODUCTLINE using Spark DataFrame?

In [110]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window


salesData = sales_df1.withColumn("year", year("ORDERDATE"))
salesData = salesData.withColumn("month", month("ORDERDATE"))


monthlySales = salesData.groupBy("year", "month", "PRODUCTLINE").agg(sum("SALES").alias("total_sales"))

windowSpec = Window.partitionBy("PRODUCTLINE").orderBy("year", "month")

monthlySales = monthlySales.withColumn("previous_month_sales", lag("total_sales").over(windowSpec))

monthlySales = monthlySales.withColumn(
    "percentage_growth",
    when(
        col("previous_month_sales").isNotNull(),
        (col("total_sales") - col("previous_month_sales")) / col("previous_month_sales") * 100
    ).otherwise(lit(None))
)

monthlySales.show()


+----+-----+-----------+------------------+--------------------+-------------------+
|year|month|PRODUCTLINE|       total_sales|previous_month_sales|  percentage_growth|
+----+-----+-----------+------------------+--------------------+-------------------+
|2003|    2|Motorcycles|25783.760000000002|                null|               null|
|2003|    3|Motorcycles|          12639.15|  25783.760000000002| -50.98019063162239|
|2003|    4|Motorcycles|23475.590000000004|            12639.15|   85.7370946622202|
|2003|    5|Motorcycles|          22097.32|  23475.590000000004|-5.8710771486467594|
|2003|    6|Motorcycles|           2642.01|            22097.32| -88.04375372217082|
|2003|    7|Motorcycles| 37924.23000000001|             2642.01|  1335.430978686682|
|2003|    8|Motorcycles|44164.909999999996|   37924.23000000001| 16.455653812878953|
|2003|    9|Motorcycles|           3155.58|  44164.909999999996| -92.85500638402749|
|2003|   10|Motorcycles| 64235.65000000001|             3155.58| 

u) How can you rebalance the data by portioning based on the COUNTRY column to ensure that
large data partitions are avoided?

In [126]:
unique_country_count = sales_df1.select("COUNTRY").distinct().count()


repartitioned_df = sales_df1.repartition(unique_country_count, "COUNTRY")


print(f"No of partitions: {repartitioned_df.rdd.getNumPartitions()}")

No of partitions: 19


v) Suppose you have a smaller lookup table with customer details. How would you perform a
broadcast join with the large sales_data_sample dataset to improve join performance? What are
the key considerations when using broadcast joins?

In [121]:

# Example data for customer details
customer_data = [
    (1, "Land of Toys Inc.", "USA"),
    (2, "Reims Collectables", "Canada"),
    (3, "Lyon Souveniers", "Mexico")
]

customer_columns = ["customer_id", "name", "country"]

customer_details = spark.createDataFrame(customer_data, customer_columns)

from pyspark.sql.functions import broadcast


broadcast_customer_details = broadcast(customer_details)


joined_df = sales_df1.join(broadcast_customer_details, 
                                   sales_df1["CUSTOMERNAME"] == broadcast_customer_details["name"])


joined_df.show()



+-----------+---------------+---------+---------------+-------+-------------------+---------+------+--------+-------+------------+----+-----------+------------------+----------------+--------------------+------------+-----+-----+----------+-------+---------+---------------+----------------+--------+----+--------+-----------+------------------+-------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|          ORDERDATE|   STATUS|QTR_ID|MONTH_ID|YEAR_ID| PRODUCTLINE|MSRP|PRODUCTCODE|      CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2| CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|YEAR|CATEGORY|customer_id|              name|country|
+-----------+---------------+---------+---------------+-------+-------------------+---------+------+--------+-------+------------+----+-----------+------------------+----------------+--------------------+------------+-----+-----+----------+-------+---------+---------------+----------------+-

w) Create a UDF that categorizes the sales values (SALES) into custom buckets like “Low”,
“Medium”, “High”. Apply this UDF to the DataFrame and calculate the count of orders in each
category per COUNTRY.

In [122]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def categorize_sales(ntile):
    if ntile == 1:
        return "Low"
    elif ntile == 2:
        return "Medium"
    else:
        return "High"

categorize_sales_udf = udf(categorize_sales, StringType())
window_spec = Window.orderBy(col("SALES"))
sales_df2 =sales_df2.withColumn("NTILE", ntile(3).over(window_spec))
df_categorized = sales_df2.withColumn('SALES_CATEGORY', categorize_sales_udf(col('NTILE')))
df_categorized.groupBy('COUNTRY', 'SALES_CATEGORY').count().show()

+-----------+--------------+-----+
|    COUNTRY|SALES_CATEGORY|count|
+-----------+--------------+-----+
|Philippines|           Low|    7|
|     Norway|        Medium|   22|
|        USA|           Low|  319|
|    Austria|           Low|   17|
|    Denmark|           Low|   18|
|     Canada|          High|   18|
|     Canada|        Medium|   24|
|     Canada|           Low|   28|
|      Italy|          High|   32|
|Switzerland|        Medium|   12|
|    Ireland|          High|    8|
|    Ireland|           Low|    7|
|Philippines|          High|    9|
|  Australia|          High|   57|
|  Singapore|           Low|   28|
|    Finland|        Medium|   34|
|     Norway|          High|   33|
|        USA|        Medium|  338|
|      Italy|           Low|   38|
|        USA|          High|  347|
+-----------+--------------+-----+
only showing top 20 rows



x) Create a Python UDF to calculate discounts for specific product lines. For example, give a
10% discount for Classic Cars and 5% for Motorcycles. Apply this UDF to derive new discounted
sales values.

In [124]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType


def calculate_discounted_sales(product_line, sales):
    if product_line == 'Classic Cars':
        discount = 0.10
    elif product_line == 'Motorcycles':
        discount = 0.05
    else:
        discount = 0.0
    return float(sales * (1 - discount))


calculate_discounted_sales_udf = udf(calculate_discounted_sales, FloatType())


sales_df2 = sales_df2.withColumn(
    'DISCOUNT_SALES',
    calculate_discounted_sales_udf(col('PRODUCTLINE'), col('SALES'))
)


sales_df2.select(col('PRODUCTLINE'), col('SALES'), col('DISCOUNT_SALES')).show()


+-----------+-------+--------------+
|PRODUCTLINE|  SALES|DISCOUNT_SALES|
+-----------+-------+--------------+
|Motorcycles| 2871.0|       2727.45|
|Motorcycles| 2765.9|      2627.605|
|Motorcycles|3884.34|      3690.123|
|Motorcycles| 3746.7|      3559.365|
|Motorcycles|5205.27|     4945.0063|
|Motorcycles|3479.76|      3305.772|
|Motorcycles|2497.77|     2372.8816|
|Motorcycles|5512.32|      5236.704|
|Motorcycles|2168.54|      2060.113|
|Motorcycles|4708.44|      4473.018|
|Motorcycles|3965.66|      3767.377|
|Motorcycles|2333.12|      2216.464|
|Motorcycles|3188.64|      3029.208|
|Motorcycles|3676.76|      3492.922|
|Motorcycles|4177.35|     3968.4824|
|Motorcycles|4099.68|      3894.696|
|Motorcycles|2597.39|     2467.5205|
|Motorcycles|4394.38|      4174.661|
|Motorcycles|4358.04|      4140.138|
|Motorcycles|4396.14|      4176.333|
+-----------+-------+--------------+
only showing top 20 rows



z) How do you implement a cumulative distribution function (CDF) over the SALES value for
each CUSTOMERNAME? What insights can you gather from analyzing the CDF distribution for
each customer?

In [142]:
window_spec = Window.partitionBy('CUSTOMERNAME').orderBy('SALES')
df_with_rank = sales_df1.withColumn('rank', row_number().over(window_spec))
df_with_count = df_with_rank.withColumn('total_count', count('SALES').over(Window.partitionBy('CUSTOMERNAME')))
df_cdf = df_with_count.withColumn('CDF', col('rank') / col('total_count'))
df_cdf = df_cdf.select('CUSTOMERNAME', 'SALES', 'CDF').show(50)

+-------------------+-------+--------------------+
|       CUSTOMERNAME|  SALES|                 CDF|
+-------------------+-------+--------------------+
|Suominen Souveniers| 891.03| 0.03333333333333333|
|Suominen Souveniers| 1086.6| 0.06666666666666667|
|Suominen Souveniers|1103.76|                 0.1|
|Suominen Souveniers|1629.04| 0.13333333333333333|
|Suominen Souveniers| 1988.4| 0.16666666666666666|
|Suominen Souveniers|2140.11|                 0.2|
|Suominen Souveniers|2447.76| 0.23333333333333334|
|Suominen Souveniers|2632.89| 0.26666666666666666|
|Suominen Souveniers| 2773.8|                 0.3|
|Suominen Souveniers|2775.08|  0.3333333333333333|
|Suominen Souveniers|2817.87| 0.36666666666666664|
|Suominen Souveniers|2851.84|                 0.4|
|Suominen Souveniers|2931.98| 0.43333333333333335|
|Suominen Souveniers|3128.65|  0.4666666666666667|
|Suominen Souveniers|3288.82|                 0.5|
|Suominen Souveniers|3595.62|  0.5333333333333333|
|Suominen Souveniers|3686.54|  

Insights

    * CDF values show how sales are distributed within each customer group.
    * Sales Concentration for a customer like "Suominen Souveniers," we can see that a significant portion of the total sales is concentrated at higher values.
    * For example, 0.7 CDF corresponds to a sales value of 4381.25, meaning that 70% of the sales data falls below this amount.



In [95]:
sales_df1.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: timestamp (nullable = true)
 |-- STATUS: string (nullable = false)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = false)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = false)
 |-- CUSTOMERNAME: string (nullable = false)
 |-- PHONE: string (nullable = false)
 |-- ADDRESSLINE1: string (nullable = false)
 |-- ADDRESSLINE2: string (nullable = false)
 |-- CITY: string (nullable = false)
 |-- STATE: string (nullable = false)
 |-- POSTALCODE: string (nullable = false)
 |-- COUNTRY: string (nullable = false)
 |-- TERRITORY: string (nullable = false)
 |-- CONTACTLASTNAME: string (nullable = false)
 |-- CONTACTFIRSTNAME: strin

In [96]:
sales_df1.show(10)

+-----------+---------------+---------+---------------+-------+-------------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+----+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|          ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|YEAR|CATEGORY|
+-----------+---------------+---------+---------------+-------+-------------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+----+--------+
|      10107|             30|     